In [40]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
from proj1_helpers import load_csv_data, predict_labels, create_csv_submission
import math

## Load data

In [42]:
y_train, input_train, ids_train = load_csv_data('train.csv', sub_sample=False)
print(y_train.shape)
print(input_train.shape)

(250000,)
(250000, 30)


In [43]:
input_train

array([[ 138.47 ,   51.655,   97.827, ...,    1.24 ,   -2.475,  113.497],
       [ 160.937,   68.768,  103.235, ..., -999.   , -999.   ,   46.226],
       [-999.   ,  162.172,  125.953, ..., -999.   , -999.   ,   44.251],
       ...,
       [ 105.457,   60.526,   75.839, ..., -999.   , -999.   ,   41.992],
       [  94.951,   19.362,   68.812, ..., -999.   , -999.   ,    0.   ],
       [-999.   ,   72.756,   70.831, ..., -999.   , -999.   ,    0.   ]])

## Pre-processing

In [44]:
def standardize_train(x):
    ''' standardize training set
    '''
    centered_data = x - np.mean(x, axis=0)
    std_data = centered_data / np.std(x, axis=0)
    
    return std_data, np.mean(x, axis=0), np.std(x, axis=0)

def standardize_test(x, mean, std):
    ''' standardize test set with same values as training set
    '''
    return (x-mean)/std

In [45]:
def add_one_column(x):
    vect_one = np.ones([x.shape[0],1])
    return np.concatenate((vect_one, x), axis = 1)

In [46]:
# IDEE: on pourrait ausi tenter build_polynomial comme dans les séries si on est motivés

In [65]:
# Standardise data
# Careful to standardize the x_test with the mean and std of x_train
x_train, mean, std = standardize_train(input_train)
x_train = add_one_column(x_train)
#x_test = standardize_test(input_test, mean, std)

In [66]:
print(x_train.shape)
#print(x_test.shape)

(250000, 31)


In [67]:
x_train

array([[ 1.        ,  0.46141372,  0.06833197, ...,  1.5668    ,
         1.55858439,  0.4125105 ],
       [ 1.        ,  0.51670419,  0.55250482, ..., -0.63936657,
        -0.63936694, -0.27381996],
       [ 1.        , -2.33785898,  3.19515553, ..., -0.63936657,
        -0.63936694, -0.29396985],
       ...,
       [ 1.        ,  0.38016991,  0.31931645, ..., -0.63936657,
        -0.63936694, -0.31701723],
       [ 1.        ,  0.35431502, -0.84532397, ..., -0.63936657,
        -0.63936694, -0.74543941],
       [ 1.        , -2.33785898,  0.66533608, ..., -0.63936657,
        -0.63936694, -0.74543941]])

## MSE method GD and SGD from lab02
I put it here to use as reference for the function to implement.

Could try it too  --> will be less good than Ridge if find good lambda

In [68]:
# Define the parameters of the algorithm. (probably change afterwards)
max_iters = 5                                                   
gamma = 0.7
batch_size = 1

# Initialization
w_initial = np.ones(x_train.shape[1])

In [69]:
w_initial.shape

(31,)

In [70]:
#From lab02 correction
# MSE GD and SGD
def calculate_mse(e):
    """Calculate the mse for vector e."""
    return np.mean(e**2)/2

def compute_loss_mse(y, tx, w):
    """Calculate the loss.

    You can calculate the loss using mse or mae.
    """
    e = y - tx.dot(w)
    return calculate_mse(e)

def compute_gradient_mse(y, tx, w):
    """Compute the gradient."""
    err = y - tx.dot(w)
    #print(err)
    #print(len(err))
    #print(-tx.T.dot(err))
    grad = -tx.T.dot(err) / len(err)
    return grad, err

def gradient_descent_mse(y, tx, initial_w, max_iters, gamma):
    """Gradient descent algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        # compute loss, gradient
        grad, err = compute_gradient_mse(y, tx, w)
        loss = calculate_mse(err)
        # gradient w by descent update
        w = w - gamma * grad
        # store w and loss
        ws.append(w)
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return losses, ws

def stochastic_gradient_descent(
        y, tx, initial_w, batch_size, max_iters, gamma):
    """Stochastic gradient descent."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w

    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient and loss
            grad, _ = compute_gradient_mse(y_batch, tx_batch, w)
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # calculate loss
            loss = compute_loss_mse(y, tx, w)
            # store w and loss
            ws.append(w)
            losses.append(loss)

        print("SGD({bi}/{ti}): loss={l}".format(bi=n_iter, ti=max_iters - 1, l=loss))
    return losses, ws

In [71]:
losses, ws = gradient_descent_mse(y_train, x_train, w_initial, max_iters, gamma)

Gradient Descent(0/4): loss=112.59444572590787, w0=0.07973519999997924, w1=-1.814929860244586
Gradient Descent(1/4): loss=5871.509948405188, w0=-0.19634424000014766, w1=16.00421250811494
Gradient Descent(2/4): loss=337846.15328975685, w0=-0.2791680719989442, w1=-119.21362055336922
Gradient Descent(3/4): loss=19447679.922442727, w0=-0.30401522160813466, w1=906.4767679639552
Gradient Descent(4/4): loss=1119484055.6777604, w0=-0.31146936641814404, w1=-6876.383695387108


## Should be useful

In [72]:
#besoin pour SGD, prise de lab02
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]      

In [73]:
# Celle du lab03, juste sans les degrés
def plot_train_test(train_errors, test_errors, lambdas):
    plt.semilogx(lambdas, train_errors, color='b', marker='*', label="Train error")
    plt.semilogx(lambdas, test_errors, color='r', marker='*', label="Test error")
    plt.xlabel("lambda")
    #plt.ylabel("RMSE")
    plt.ylabel("Loss")
    plt.title("Ridge regression for polynomial degree ")
    leg = plt.legend(loc=1, shadow=True)
    leg.draw_frame(False)
    #plt.savefig("ridge_regression")

# Methods to implement 
P.2 of project description

-  least_square
-  least_square_GD
-  least_square_SGD

-  ridge_regression
-  logistic_regression
-  reg_logistic_regression

All function should return (w, loss) <br>
Later add them in implementations.py

In [197]:
# Define the parameters of the algorithm. (probably change afterwards)
max_iters = 200                                          
gamma = 0.1
batch_size = 1

# Initialization
w_initial = np.random.rand(x_train.shape[1])

## Least square 

In [198]:
def compute_loss_ls(y, tx, w):
    """Calculate the loss.
    """
    e = y - tx.dot(w)
    #print(e)
    return (1/2)*np.mean(e**2)

In [199]:
e = compute_loss_ls(y_train, x_train, w_initial)
#print(e)

In [200]:
def least_squares(y, tx):
    """Compute the optimal w and the loss with least square technique"""
    """Required by project description"""
    a = tx.T.dot(tx) 
    b = tx.T.dot(y)
    w = np.linalg.solve(a, b)                #p.7 du cours least squares
    loss = compute_loss_ls(y, tx, w)                 #p.3 du cours least squares
    return w, loss

In [201]:
w_ls, loss_ls = least_squares(y_train, x_train)
print("w is {} \n of shape {}".format(w_ls, w_ls.shape))
print("\n loss of least squares is {}".format(loss_ls))

w is [-3.14664000e-01  2.93788272e-02 -2.52531475e-01 -2.54791124e-01
 -3.03696824e-02 -1.40144726e+00  2.95701641e-01 -1.07889472e+01
  2.67880862e-01 -2.44934971e-03 -3.28827529e+02 -1.82647888e-01
  1.14039627e-01  2.05045961e+01  6.38853340e+01 -3.18961885e-04
 -1.80884297e-03  6.29944860e+01 -4.48641524e-04  1.54379296e-03
  1.21462701e-01  3.95268757e-04 -6.33223475e-02 -2.06747093e-01
 -1.16655767e-01  9.86256328e-02  1.67907718e-01 -3.35146253e-02
 -2.98358685e+00 -5.36388099e+00  2.78478918e+02] 
 of shape (31,)

 loss of least squares is 0.33944559848933104


In [202]:
def compute_gradient_least_squares(y, tx, w):
    """Compute the gradient of least square."""
    '''
    print("y is {}".format(y.shape))
    print("tx is {}".format(tx.shape))
    print("w is {}".format(w.shape))
    a = tx.dot(w)
    print("a is {}".format(a.shape))
    '''
    err = y - w.dot(tx.T) #tx.dot(w)
    #print(err)
    #print(err.shape)
    grad = -tx.T.dot(err)/len(err)         #p.5 du cours least squares
    return grad                  

In [203]:
#compute_gradient_least_square(y_train, x_train, w)
grad_ls = compute_gradient_least_squares(y_train, x_train, w_initial)
print("grad is {} \n of shape {}".format(grad_ls, grad_ls.shape))
#print("\nerr of least squares is {}".format(err))

grad is [ 0.82925627  1.35486047 -0.53388776  0.94381044  4.14668106  4.38643816
  4.18672509  4.38796294 -1.70062676  2.54162102  4.84979533  0.4515363
  3.09465883  4.38704467  2.11892879  1.32081617  0.78263535  1.92736598
  1.28733905  0.52094746  2.58049624  0.74010854  4.69893771  5.15819561
  4.67225708  4.51522475  4.51327119  4.41997267  4.38746095  4.38681567
  4.8067236 ] 
 of shape (31,)


In [204]:
gamma = 0.1
def least_squares_GD(y, tx, w_initial, max_iters, gamma):
    """Gradient descent algorithm with least squares."""
    """Required by project description"""
    w = w_initial
    for n_iter in range(max_iters):
        # compute gradient and error
        grad = compute_gradient_least_squares(y, tx, w)
        #print(grad)
        # gradient w by descent update
        w = w - (gamma * grad)
        # print(w)
        # calculate loss    
    loss = compute_loss_ls(y, tx, w)           
    return w, loss

In [205]:
w_ls_gd, loss_ls_gd = least_squares_GD(y_train, x_train, w_initial, max_iters, gamma)
print("w is {} \n of shape {}".format(w_ls_gd, w_ls_gd.shape))
print("\n loss of least squares is {}".format(loss_ls_gd))

w is [-0.314664    0.03551621 -0.2632791  -0.23753145 -0.04211536 -0.08903241
  0.38741625 -0.16497864  0.24720383 -0.02052449 -0.22774113 -0.12825728
  0.1206671  -0.02182477  0.26182533 -0.00059858 -0.00106568  0.28433094
 -0.00091532  0.0027397   0.13953565  0.00115236 -0.0302467  -0.04043206
 -0.02335896 -0.09567177  0.20069583  0.19775776 -0.13618977 -0.00642423
  0.02321595] 
 of shape (31,)

 loss of least squares is 0.3410296670896865


In [221]:
gamma = 0.01
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    """Stochastic Gradient Descent algorithm with least squares."""
    """Required by project description"""
    w = initial_w

    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient and loss
            grad = compute_gradient_least_squares(y_batch, tx_batch, w)
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # calculate loss
    loss = compute_loss_ls(y, tx, w)                                 #TO CHECK p.3 least squares
    return w, loss

In [222]:
w_ls_sgd, loss_ls_sgd = least_squares_SGD(y_train, x_train, w_initial, max_iters, gamma)
print("w is {} \n of shape {}".format(w_ls_sgd, w_ls_sgd.shape))
print("\n loss of least squares is {}".format(loss_ls_sgd))

w is [-0.37166463 -0.00632278  0.1514792  -0.14046812 -0.34975606 -0.07375509
  0.17958506 -0.14408062  0.23625826  0.11247356 -0.19644417 -0.06826166
  0.31921351 -0.00507783  0.38774907  0.02588742  0.07996289  0.09645135
  0.09720648 -0.06592488  0.142341    0.03936843  0.4962332   0.14426907
 -0.08692388 -0.15036215  0.14488584  0.21884451 -0.11744068  0.01120955
  0.07366224] 
 of shape (31,)

 loss of least squares is 0.567972586721236


## Ridge regression

In [208]:
# Should do grid search  TO DO
lambda_ = 0.01

In [209]:
##################
# Je suis un peu confuse. Est ce que ce qu'il veut c'est ça ou bien ce que j'ai fait aux fonctions ridge_GD et ridge_SGD ?? 
# ridge_GD et ridge_SGD sont juste en-dessous
def ridge_regression(y, tx, lambda_):
    """Ridge regression."""
    """Required by project description"""
    lambd = 2 * tx.shape[0] * lambda_
    a = tx.T.dot(tx) + lambd * np.identity(tx.shape[1])
    b = tx.T.dot(y) 
    w = np.linalg.solve(a, b) 
    err = y - tx.dot(w)
    loss = (1/2) * np.mean(err**2) + lambda_ * (np.linalg.norm(w,2))**2   #TO CHECK p.3 ridge regression
    return w, loss

In [210]:
w_rreg, loss_rreg = ridge_regression(y_train, x_train, lambda_)
print("w is {} \n of shape {}".format(w_rreg, w_rreg.shape))
print("\n loss of ridge regression is {}".format(loss_rreg))

w is [-0.30849412  0.03572402 -0.24077855 -0.21610016 -0.01054923 -0.01919922
  0.34814992 -0.03018983  0.23504977 -0.01013674 -0.00087247 -0.15674991
  0.11455458 -0.02161131  0.18503225 -0.00076269 -0.00130455  0.23850622
 -0.00086979  0.00249252  0.10383837  0.00113401 -0.06198121 -0.14991608
  0.03645409  0.04367213  0.0436772  -0.02328488 -0.02309077 -0.0234825
 -0.09703028] 
 of shape (31,)

 loss of ridge regression is 0.3463797801150265


In [211]:
def compute_gradient_ridge(y, tx, w, lambda_):
    """Not required specifically""" 
    err = y - tx.dot(w)
    grad = -tx.T.dot(err) / len(err) + (2 * lambda_* w)
    return grad, err

In [212]:
gamma = 0.1
def ridge_GD(y, tx, initial_w, max_iters, gamma, lambda_):
    """Gradient descent algorithm with Ridge."""
    """Not required specifically""" 
    w = initial_w
    for n_iter in range(max_iters):
        # compute gradient and error
        grad, _ = compute_gradient_ridge(y, tx, w, lambda_)
        #print("grad is {} \n of shape {}".format(grad, grad.shape))
        #print("\n err of least squares is {}".format(err))
        # gradient w by descent update
        w = w - gamma * grad
        # calculate loss
    loss = 1/2*np.mean((y - tx.dot(w))**2)            
    return w, loss

In [213]:
w, loss = ridge_GD(y_train, x_train, w_initial, max_iters, gamma, lambda_)
print("w is {} \n of shape {}".format(w, w.shape))
print("\n loss of ridge_GD is {}".format(loss))

w is [-0.30849412  0.03969903 -0.24803349 -0.20668975 -0.02071613 -0.06276211
  0.33439195 -0.11527076  0.22283479 -0.0216681  -0.15269138 -0.1237026
  0.11820039 -0.01815791  0.23046428 -0.00072156 -0.00128755  0.24036147
 -0.00088598  0.00262998  0.11749501  0.00121341 -0.0389671  -0.05794899
 -0.00814204 -0.05584488  0.1427407   0.12822726 -0.09492894 -0.00800047
 -0.00835744] 
 of shape (31,)

 loss of ridge_GD is 0.3416704527539797


In [223]:
gamma = 0.01
def ridge_SGD(y, tx, initial_w, max_iters, gamma, lambda_):
    """Stochastic Gradient Descent algorithm with least squares."""
    """Not required specifically""" #Implicitely requested I think
    
    w = initial_w
    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient and loss
            grad, _ = compute_gradient_ridge(y_batch, tx_batch, w, lambda_)    
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # calculate loss
    loss = np.mean((y - tx.dot(w))**2)/2 + lambda_ * (np.linalg.norm(w,2))**2
    return w, loss

In [224]:
w, loss = ridge_SGD(y_train, x_train, w_initial, max_iters, gamma, lambda_)
print("w is {} \n of shape {}".format(w, w.shape))
print("\n loss of ridge SGD is {}".format(loss))

w is [-0.16088127 -0.08597774  0.06764747 -0.72758024 -0.21006235 -0.01543221
  0.2134124  -0.08507768  0.32333488  0.2422295  -0.08728427  0.147203
  0.35983842  0.0499339   0.17484606 -0.03415117  0.18109739  0.18260342
  0.04354737  0.09384829  0.11395921  0.01771609  0.61530534  0.21592138
 -0.04652917 -0.09264811  0.1921234   0.27806457 -0.05857891  0.06558203
  0.21550393] 
 of shape (31,)

 loss of ridge SGD is 1.472402904853191


In [115]:
lambdas = np.logspace(-4, -0.05, 15)

rmse_tr = []
rmse_te = []
for ind, lambda_ in enumerate(lambdas):
    w, loss_tr = ridge_SGD(y_train, x_train, w_initial, max_iters, gamma, lambda_)
    rmse_tr.append(np.sqrt(2*loss_tr))
    loss_te = (1/2) * np.mean((y_test - x_test.dot(w))**2) + lambda_ * (np.linalg.norm(w,2))**2
    rmse_te.append(np.sqrt(2*loss_te))

KeyboardInterrupt: 

In [ ]:
# Grid search to choose lambda
plot_train_test(rmse_tr, rmse_te, lambdas)

## Logistic regression
Debugged but not sure from here: Il y aura probablement une correction de labo pour améliorer / re-vérifier les fonctions de logistic regression

### Il y a un NaN dans la loss à cause d'un outlier. Je sais pas si on choisis d'enlever ou comment traiter. On verra.

In [228]:
def sigmoid(tx, w):
    """Compute sigmoid function"""
    z = np.array(np.exp(-tx.dot(w)))
    return 1./(1 + z)

value = sigmoid(x_train, w_initial)
print(value)

[0.99981039 0.90820056 0.92786083 ... 0.28142199 0.00709906 0.04907172]


In [226]:
def compute_logreg_loss(y, tx, w):  #np.log parce que math.log fonctionne pas.. J'ai toujours pas compris pourquoi.
    """Compute error and gradient of logistic regression"""
    sig = sigmoid(tx, w)
    loss = np.sum((-y * np.log(sig) - (1-y) * np.log(1-sig)), axis = -1)/len(y)
    print(max(sig))
    #a = (-y * np.log(sig) - (1-y) * np.log(1-sig))
    #print(max(a))
    return loss

In [227]:
loss = compute_logreg_loss(y_train, x_train, w_initial)
print("loss is {}".format(loss))

[0.51459227 0.10073464 0.53488351 0.42669814 0.02003378 0.140064
 0.2264286  0.072293   0.49416143 0.6004996  0.14155126 0.00350222
 0.81835964 0.2093738  0.7436304  0.88667924 0.88977018 0.11193385
 0.70287969 0.70028424 0.34716633 0.66417114 0.94782936 0.73171242
 0.5700489  0.53273724 0.82729083 0.43374625 0.09599914 0.22535177
 0.38780099]
1.0
loss is nan


C:\Users\Pauli\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
C:\Users\Pauli\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in multiply
  after removing the cwd from sys.path.


In [119]:
def compute_logreg_grad(y, tx, w):
    """Compute error and gradient of logistic regression""" 
    sig = sigmoid(tx, w)
    err  = sig - y
    grad = tx.T.dot(err)/len(y)
    return grad

In [ ]:
grad_lr = compute_logreg_grad(y_train, x_train, w_initial)
print("grad is {} \n of shape {}".format(grad_lr, grad_lr.shape))

In [ ]:
gamma = 0.01
def logistic_regression(y, tx, initial_w, max_iters, gamma): #SGD  (GD easy to implement from here)
    """Stochastic Gradient Descent algorithm with logistic regression."""
    """Required by project description"""
    w = initial_w

    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient
            grad = compute_logreg_grad(y_batch, tx_batch, w)
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # compute a stochastic loss
            compute_logreg_loss(y_batch, tx_batch, w)
    return w, loss

In [ ]:
w_lr, loss_lr = logistic_regression(y_train, x_train, w_initial, max_iters, gamma)
print("w is {} \n of shape {}".format(w_lr, w_lr.shape))
print("\n loss of ridge_SGD is {}".format(loss_lr))

## Regularized Logistic Regression

In [ ]:
def compute_logreg_reg_loss(y, tx, w, lambda_):
    """Compute error and gradient of logistic regression"""
    reg = ( lambda_/(2*len(y)) ) * sum(w**2)
    loss = compute_logreg_loss(y, tx, w) + reg
    return loss

In [ ]:
loss_lrr = compute_logreg_reg_loss(y_train, x_train, w_initial, lambda_)
print("loss is {}".format(loss_lrr))

In [ ]:
def compute_logreg_reg_grad(y, tx, w, lambda_):
    """Compute error and gradient of logistic regression"""     
    grad = compute_logreg_grad(y, tx, w) 
    reg = (lambda_/len(y)) * w[1:]
    grad[1:] = grad[1:] + reg            
    return grad

In [ ]:
grad_lrr = compute_logreg_reg_grad(y_train, x_train, w_initial, lambda_)
print("grad is {} \n of shape {}".format(grad_lrr, grad_lrr.shape))

In [ ]:
###########
# La loss de ridge SGD me semble particulièrement haute. A vérifier avec les plots (et/ou code).
gamma = 0.01
def reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma):
    """Stochastic Gradient Descent algorithm with REGULARIZED logistic regression."""
    """Required by project description"""
    w = initial_w

    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient
            grad = compute_logreg_reg_grad(y_batch, tx_batch, w, lambda_)
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # compute a stochastic loss
            loss = compute_logreg_reg_loss(y_batch, tx_batch, w, lambda_)
    return w_rlr_sgd, loss_rlr_sgd

In [ ]:
w_rlr_sgd, loss_rlr_sgd = reg_logistic_regression(y_train, x_train, lambda_, w_initial, max_iters, gamma)
print("w is {} \n of shape {}".format(w_rlr_sgd, w_rlr_sgd.shape))
print("\n loss of ridge_SGD is {}".format(loss_rlr_sgd))

# Prediction de chaque méthode

In [ ]:
#y_pred = predict_labels(weights, data)

## TODO: Grid search pour meilleure gamma et lambda pour chaque méthode (pour lambda faire en meme temps que lambda (influence)

## Choisir méthode:
(D'après moi mais hésitez pas à faire autrement si vous avez une meilleure idée)
- Pour chaque méthode: faire un plot de test_error (=loss) pour un grand nombre d'iteration (assez pour que on voit que l'erreur commence à remonter si possible).  Il faudrait idealement voir la courbe descendre et monter dans le graphe (underfit à overfit)
- Garder la valeur de n iter ou la test error était minimale (au cas où: risque d'être très long pour ridge si lambda_ trop grand)
- Comparer les loss de chaque méthode à leur n_iter optimale
- La meilleure est celle ou c'est le plus faible

En plus comme ça on aura des beau plots et des arguments pour le rapport.